# Run SHAP

This file is part of the Comparative analysis of tissue-specific genes in maize based on machine learning models: CNN performs technically best, LightGBM performs biologically sound project.

### Objective:
> Load trained model and calculate SHAP values for test data set

### Input files:
1. *Filtered_Maize_expression.csv*
3. *{data_type}_model_topology.json*
4. *{data_type}_model_weights.hdf5*

### Output files:
1. *shap_scores_{data_type}.pkl* 
2. *{data_type}_ranks.pkl* 
3. *shap_genes.pkl* 


### Table of contents:
1. [Import Modules](#1.-Import-Modules)  
2. [Set static paths](#2.-Set-static-paths)  
3. [Load files](#3.-Load-files)  
    3.1 [Load RNAseq](#3.1-Load-RNAseq)  
    3.2 [Load model](#3.2-Load-model)  
4. [Run SHAP model](#4.-Run-SHAP-model)  
    4.1 [Run inference](#4.1-Run-inference)  
    4.2 [Fit SHAP](#4.2-Fit-SHAP)  
    4.3 [Get SHAP values](#4.3-Get-SHAP-values)  
    4.4 [Filter SHAP values](#4.4-Filter-SHAP-values)  
    4.5 [Rank SHAP values](#4.5-Rank-SHAP-values)  
    4.6 [Get unique genes](#4.6-Get-unique-genes)  
5. [Save out SHAP scores](#5.-Save-out-SHAP-scores) 

## 1. Import Modules

In [1]:
import shap
import pickle
import numpy as np
import pandas as pd

In [2]:
Maize_expression = pd.read_csv('2Filtered_Maize_expression.csv', index_col=0)

In [3]:
Maize_expression.head()

,Zm00001d032396,Zm00001d032398,Zm00001d032399,Zm00001d032400,Zm00001d032401,Zm00001d032402,Zm00001d032405,Zm00001d032407,Zm00001d032408,Zm00001d032409,...,Zm00001d045391,Zm00001d045361,Zm00001d045063,Zm00001d045054,Zm00001d045015,Zm00001d044966,Zm00001d046033,Zm00001d046016,Zm00001d047658,tissue
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SRR1030974,8.157619,0.0,2.275141,52.605335,0.514260,58.711246,47.712910,14.526341,0.591568,7.529287,...,2.773152,5.296510,3.277321,0.298473,2.080474,5.034541,6.040693,42.783398,0.196235,root
SRR1030975,7.264478,0.0,2.779206,57.824116,0.559446,52.145851,82.328278,13.596546,0.364540,3.799316,...,6.606791,4.113046,1.851271,0.261718,2.769382,5.007706,5.638885,41.406376,0.000000,root
SRR1030976,8.328547,0.0,3.230831,42.953331,0.334829,49.397705,49.869648,8.037382,0.405711,6.947237,...,38.602425,2.604917,4.993490,1.731227,2.359700,2.586420,6.572925,51.332336,0.109857,root
SRR1030977,6.621617,0.0,2.838485,65.223167,1.483768,53.151070,75.148918,14.176978,0.790322,3.839634,...,4.471345,10.553447,1.957708,0.029638,2.777192,4.330374,5.348520,40.181637,0.022587,root
SRR1030978,8.687157,0.0,2.788881,52.362190,0.346632,46.062824,51.403378,8.094249,0.510341,6.406972,...,111.361015,3.941180,5.267010,1.550525,2.630184,3.084651,6.505562,50.188221,0.217275,root


In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(Maize_expression,test_size = 0.2)

In [5]:
import os
util_path = '../src'
os.chdir(util_path)

In [6]:
import keras
import tensorflow
print(tensorflow.keras.__version__)

Using TensorFlow backend.


2.3.0-tf


In [7]:
from tqdm import tqdm
from tensorflow.keras import backend
from tensorflow.keras.models import model_from_json

from constant import map_dict, inv_map
from modelling.cnn import run_inference, prepare_x_y 
from shap_utils import filter_shap, get_rank_df

In [9]:
%load_ext autoreload
%autoreload 2

## 2. Set static paths

In [10]:
data_type = "imbalanced"
data_type1 = "smote"
data_dir = "../data/"

In [11]:
input_dir = data_dir + "processed/"
model_dir = f"../models/"
shap_dir = data_dir + "shap/"
gene_dir = data_dir + "gene_lists/"

## 3. Load files

#### 3.1 Load RNAseq

In [12]:
ref_data = train
test_data = test

#### 3.2 Load model

In [15]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

2.2.0
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-10-17 15:10:30.446272: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-10-17 15:10:30.457260: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA TITAN Xp computeCapability: 6.1
coreClock: 1.582GHz coreCount: 30 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 510.07GiB/s
2022-10-17 15:10:30.457502: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-10-17 15:10:30.459565: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-10-17 15:10:30.461196: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-10-17 15:10:30.461502: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10


In [16]:
# Load model beatifully
model_json_path = model_dir+f"{data_type}_model_topology.json"
model = model_from_json(
    open(model_json_path, "r").read()
)

# load weights into new model
model_weights_path = model_dir+f"{data_type}_model_weights.hdf5"
model.load_weights(model_weights_path)

2022-10-17 15:10:30.654961: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-10-17 15:10:30.665882: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2399995000 Hz
2022-10-17 15:10:30.669621: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc2d8000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-10-17 15:10:30.669650: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-10-17 15:10:31.006281: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5617904ee2e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-10-17 15:10:31.006319: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN Xp, Compute Capability 6.1
2022-10-17 15:10:31.007655: I tensorflow/core/co

## 4. Run SHAP model

#### 4.1 Run inference

In [17]:
y_pred = run_inference(test_data, model)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


2022-10-17 15:10:33.687315: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-10-17 15:10:33.879073: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


In [18]:
X_ref, _ = prepare_x_y(ref_data, 'tissue')
X_test, _ = prepare_x_y(test_data,'tissue')

#### 4.2 Fit SHAP

In [19]:
explainer = shap.GradientExplainer(model, X_ref)

#### 4.3 Get SHAP values

In [20]:
out_list = []
num_samples = np.shape(X_test)[0]
for sample in tqdm(range(num_samples)):
    # shap
    shap_values = explainer.shap_values(X_test[sample : sample + 1])
    out_list.append(shap_values)
shap_arr = np.squeeze(np.array(out_list))

100%|██████████| 191/191 [19:18<00:00,  6.06s/it]


#### 4.4 Filter SHAP values

In [21]:
test_data

,Zm00001d032396,Zm00001d032398,Zm00001d032399,Zm00001d032400,Zm00001d032401,Zm00001d032402,Zm00001d032405,Zm00001d032407,Zm00001d032408,Zm00001d032409,...,Zm00001d045391,Zm00001d045361,Zm00001d045063,Zm00001d045054,Zm00001d045015,Zm00001d044966,Zm00001d046033,Zm00001d046016,Zm00001d047658,tissue
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SRR13178458,1.801932,0.000000,0.000000,0.479180,0.000000,8.661842,18.073269,1.567577,0.000000,0.188149,...,0.718594,0.000000,1.703259,0.000000,0.000000,0.000000,0.796752,10.920719,0.197977,root
SRR822414,6.580391,0.000000,3.269700,76.652313,1.897456,27.177441,85.541176,11.323411,3.556690,5.307981,...,0.070374,3.953855,38.321037,0.229835,1.488381,2.423226,9.357058,60.576412,258.383057,root
SRR7469522,1.039182,0.000000,4.224452,8.362391,3.674899,11.449785,0.281489,1.417132,0.179082,0.623996,...,2.485956,14.502257,0.000000,0.000000,1.570850,2.229773,1.272122,4.742080,0.000000,ligule
SRR13332803,1.347539,0.000000,1.883695,1.121357,0.000000,21.228218,0.427153,0.145446,0.218195,0.681272,...,0.000000,2.066655,0.430612,0.000000,0.000000,0.000000,3.174280,17.252279,0.000000,leaf
SRR7469486,6.466619,0.000000,24.984127,53.351982,0.363322,14.241838,17.455805,3.248281,0.000000,0.000000,...,2.106160,3.927475,0.000000,0.000000,0.261801,0.141121,2.589484,10.738862,0.000000,blade
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR5888351,8.538136,0.000000,15.310797,30.417059,0.000000,32.389580,0.000000,0.000000,0.000000,2.470715,...,0.000000,3.737626,0.422236,3.289751,4.098902,0.000000,16.873011,17.707413,0.000000,leaf
SRR5888341,9.765476,0.000000,33.508163,16.595776,0.000000,25.093435,0.000000,0.093267,0.064208,0.361834,...,0.000000,0.313144,0.248523,68.255211,8.469850,0.000000,11.108725,12.237291,0.040230,leaf
SRR13014432,4.549399,0.011066,2.716624,106.579002,0.076796,22.229654,0.000000,0.175876,0.276533,4.221416,...,0.000000,20.299637,9.755845,0.998170,5.055567,0.116467,4.353043,38.421211,0.000000,leaf


In [22]:
shap_df = filter_shap(test_data, shap_arr, y_pred)

100%|██████████| 191/191 [00:00<00:00, 466.04it/s]
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [23]:
backend.clear_session()

#### 4.5 Rank SHAP values

In [24]:
rank_df = get_rank_df(shap_df)

#### 4.6 Get unique genes

In [25]:
gene_list = []
for index, row in tqdm(rank_df.iterrows()):
    gene_list.extend(list(row.values))
    val = len(np.unique(gene_list))/len(gene_list)
    if val <= 0.5:
        print(index)
        break

2485it [00:17, 144.83it/s]

2485


In [26]:
shap_genes = np.unique(rank_df[:index].values.flatten())

## 5. Save out SHAP scores

In [27]:
output_file = shap_dir + f"shap_scores_{data_type1}.pkl"

pickle.dump(shap_df, open(str(output_file), "wb"))

In [28]:
output_file = shap_dir + f"{data_type1}_ranks.pkl"

pickle.dump(rank_df, open(str(output_file), "wb"))

In [29]:
output_file = gene_dir + f"shap_genes.pkl"

pickle.dump(shap_genes, open(str(output_file), "wb"))